In [2]:
import pandas as pd

customer_data = pd.read_csv('data/customer_data.csv')
customer_data.head()
#todo add business name eg James William Agriculutre - later


,Customer Name,uid,business sector,location,phone number,national ID,passport,tin,gender,education level,business start date
0,James William,cca67301-ef3c-4496-b8fd-31847b7edd88,Agriculture,Fort Portal,256723196874,78602628,U6482628,8900454,Female,University,2013-02-25
1,James Nansukusa,7a1c5730-b8e7-4774-8746-90a317b73a82,Services,Kampala,256735895263,36028682,U1006980,7442154,Female,University,2009-02-22
2,Nansukusa Sarah,fb94fff4-9217-470f-9e80-12621dea6191,Fisheries,Lira,256790291092,22644880,U3152513,2102298,Male,Vocational,2009-02-07
3,Bridget Namwalwa,ff0563dc-6006-4b4f-a88a-1e7b64adb040,Manufacturing,Kisoro,256773437304,78654217,U3734663,9738952,Male,University,2016-04-15
4,William William,a185cde9-4893-42c0-bcad-fbcf101d2a77,Hospitality,Kampala,256712698533,71758895,U4129268,4037967,Male,Primary,2013-03-15


In [3]:
transactions_data = pd.read_csv('data/transactions_data.csv')
transactions_data.head()

,Customer Name,uid,amount,type,reason,date
0,James William,cca67301-ef3c-4496-b8fd-31847b7edd88,70932.51,Debit,Refund,2023-07-15
1,James William,cca67301-ef3c-4496-b8fd-31847b7edd88,630576.83,Credit,Sale,2019-01-15
2,James William,cca67301-ef3c-4496-b8fd-31847b7edd88,984729.90,Credit,Purchase,2020-05-17
3,James William,cca67301-ef3c-4496-b8fd-31847b7edd88,456349.56,Credit,Sale,2022-09-30
4,James William,cca67301-ef3c-4496-b8fd-31847b7edd88,754581.57,Credit,Miscellaneous,2023-03-04


In [4]:
print(f"Number of rows in transactions_data: {transactions_data.shape[0]}")

Number of rows in transactions_data: 89227


# Data cleaning
- Remove columns from customer_data df which are not needed for our analysis
- Add a new column to customer_data which shows the business duration
- Add new columsn which calculate the value: some value containng business age and number and amount of transactions (maybe business_score?)

In [5]:

customer_data_cleaned = customer_data.drop(columns=['phone number', 'national ID', 'passport', 'tin'])
customer_data_cleaned.head()

,Customer Name,uid,business sector,location,gender,education level,business start date
0,James William,cca67301-ef3c-4496-b8fd-31847b7edd88,Agriculture,Fort Portal,Female,University,2013-02-25
1,James Nansukusa,7a1c5730-b8e7-4774-8746-90a317b73a82,Services,Kampala,Female,University,2009-02-22
2,Nansukusa Sarah,fb94fff4-9217-470f-9e80-12621dea6191,Fisheries,Lira,Male,Vocational,2009-02-07
3,Bridget Namwalwa,ff0563dc-6006-4b4f-a88a-1e7b64adb040,Manufacturing,Kisoro,Male,University,2016-04-15
4,William William,a185cde9-4893-42c0-bcad-fbcf101d2a77,Hospitality,Kampala,Male,Primary,2013-03-15


Add business duration column

In [6]:
from datetime import datetime

#convert the business start date to data time so that we can calculate year
customer_data['business start date'] = pd.to_datetime(customer_data['business start date'])

duration_in_days = (datetime.now() - customer_data['business start date']).dt.days

duration_in_years = (duration_in_days / 365).round(2)

customer_data_cleaned['business duration (years)'] = duration_in_years

customer_data_cleaned.head()


,Customer Name,uid,business sector,location,gender,education level,business start date,business duration (years)
0,James William,cca67301-ef3c-4496-b8fd-31847b7edd88,Agriculture,Fort Portal,Female,University,2013-02-25,10.63
1,James Nansukusa,7a1c5730-b8e7-4774-8746-90a317b73a82,Services,Kampala,Female,University,2009-02-22,14.64
2,Nansukusa Sarah,fb94fff4-9217-470f-9e80-12621dea6191,Fisheries,Lira,Male,Vocational,2009-02-07,14.68
3,Bridget Namwalwa,ff0563dc-6006-4b4f-a88a-1e7b64adb040,Manufacturing,Kisoro,Male,University,2016-04-15,7.49
4,William William,a185cde9-4893-42c0-bcad-fbcf101d2a77,Hospitality,Kampala,Male,Primary,2013-03-15,10.58


### Add performance index column

Custom formula for *perfomance index*:

Perfomance Index =  (λ1 * Net Profit Per Year +  λ2 * Lifetime Net Profit + λ3 * Transaction Volume Per Year)


Where:

- λ1 is the weight for Net Profit Per Year
- λ2 is the weight for Transaction Volume Per Year
- λ3 is the weight for Lifetime Net Profit


In [7]:
# Calculate Net Profit
total_credit = transactions_data[transactions_data['type'] == 'Credit'].groupby('uid')['amount'].sum()
total_debit = transactions_data[transactions_data['type'] == 'Debit'].groupby('uid')['amount'].sum()
net_profit = total_credit.subtract(total_debit, fill_value=0)

# Calculate Transaction Volume
transaction_volume = transactions_data.groupby('uid').size()

# Merge net profit and transaction volume with customer_data on uid
customer_data_cleaned_with_net_profit = customer_data_cleaned.merge(net_profit.rename('Net Profit'), on='uid', how='left').fillna(0)
customer_data_cleaned_with_txn_vol = customer_data_cleaned_with_net_profit.merge(transaction_volume.rename('Transaction Volume'), on='uid', how='left').fillna(0)

# Calculate net profit per year and transaction volume per year
customer_data_cleaned_with_txn_vol['Net Profit Per Year'] = customer_data_cleaned_with_txn_vol['Net Profit'] / customer_data_cleaned_with_txn_vol['business duration (years)']
customer_data_cleaned_with_txn_vol['Transaction Volume Per Year'] = customer_data_cleaned_with_txn_vol['Transaction Volume'] / customer_data_cleaned_with_txn_vol['business duration (years)']

# Weights
λ1, λ2, λ3 = 1, 1, 1

# Calculate perfomance index considering lifetime profits
customer_data_cleaned_with_txn_vol['performance index'] = (λ1 * customer_data_cleaned_with_txn_vol['Net Profit Per Year'] + 
                                  λ2 * customer_data_cleaned_with_txn_vol['Transaction Volume Per Year'] + 
                                  λ3 * customer_data_cleaned_with_txn_vol['Net Profit'])

# Normalize the perfomance index: 
min_val = customer_data_cleaned_with_txn_vol['performance index'].min()  
max_val = customer_data_cleaned_with_txn_vol['performance index'].max()
customer_data_cleaned_with_txn_vol['Normalized Performance Index'] = (customer_data_cleaned_with_txn_vol['performance index'] - min_val) / (max_val - min_val)

customer_data_cleaned_with_txn_vol.head()


,Customer Name,uid,business sector,location,gender,education level,business start date,business duration (years),Net Profit,Transaction Volume,Net Profit Per Year,Transaction Volume Per Year,performance index,Normalized Performance Index
0,James William,cca67301-ef3c-4496-b8fd-31847b7edd88,Agriculture,Fort Portal,Female,University,2013-02-25,10.63,3513483.23,294,3.305252e+05,27.657573,3.844036e+06,0.183499
1,James Nansukusa,7a1c5730-b8e7-4774-8746-90a317b73a82,Services,Kampala,Female,University,2009-02-22,14.64,-3204307.06,107,-2.188734e+05,7.308743,-3.423173e+06,0.156887
2,Nansukusa Sarah,fb94fff4-9217-470f-9e80-12621dea6191,Fisheries,Lira,Male,Vocational,2009-02-07,14.68,-3393780.01,265,-2.311839e+05,18.051771,-3.624946e+06,0.156148
3,Bridget Namwalwa,ff0563dc-6006-4b4f-a88a-1e7b64adb040,Manufacturing,Kisoro,Male,University,2016-04-15,7.49,-8005741.43,262,-1.068857e+06,34.979973,-9.074564e+06,0.136193
4,William William,a185cde9-4893-42c0-bcad-fbcf101d2a77,Hospitality,Kampala,Male,Primary,2013-03-15,10.58,-8434974.90,260,-7.972566e+05,24.574669,-9.232207e+06,0.135615


# Encoding
Transforming data into machine readable numbers to facilitate the analysis process. 

## Define value we want encode
Business Sector, Location, Gender, Education level, business age, some value containng business age and number and amount of transactions

The features we choose should be abe to describe groups of rows (as compated to geatures like phone number)

## Creating dummy varialbes/ encoding
Creating numerical values from categorical features can be approached in several ways. 
The easiest is assigning a value to each value of a features. e.g 
 Business Sector
    - Agriculture : 1
    - Service : 2
    - Fisheries : 3
However, this is not effecient for clustering algorithms as it will give inaccurate clusters

A better approach is the use of one-hot encoding, where a value is 1 and only 1 possibilty, while the rest are 0


In [8]:
categorical_columns = ['business sector', 'location', 'gender', 'education level']
customer_data_encoded = pd.get_dummies(customer_data_cleaned_with_txn_vol, columns=categorical_columns)

# Force convert all dummy variables to int
categorical_dummies = [col for col in customer_data_encoded.columns if any(prefix in col for prefix in categorical_columns)]
customer_data_encoded[categorical_dummies] = customer_data_encoded[categorical_dummies].astype(int)

customer_data_encoded = customer_data_encoded.drop(columns = ["performance index"])
customer_data_encoded.head()

# Standardize Continuous Columns fro ML
# from sklearn.preprocessing import StandardScaler
# continuous_columns = ['business duration (years)', 'Transaction Volume', 'Net Profit']
# scaler = StandardScaler()
# customer_data_encoded[continuous_columns] = scaler.fit_transform(customer_data_encoded[continuous_columns])




,Customer Name,uid,business start date,business duration (years),Net Profit,Transaction Volume,Net Profit Per Year,Transaction Volume Per Year,Normalized Performance Index,business sector_Agriculture,...,location_Kisoro,location_Lira,location_Mbale,location_Mbarara,gender_Female,gender_Male,education level_Primary,education level_Secondary,education level_University,education level_Vocational
0,James William,cca67301-ef3c-4496-b8fd-31847b7edd88,2013-02-25,10.63,3513483.23,294,3.305252e+05,27.657573,0.183499,1,...,0,0,0,0,1,0,0,0,1,0
1,James Nansukusa,7a1c5730-b8e7-4774-8746-90a317b73a82,2009-02-22,14.64,-3204307.06,107,-2.188734e+05,7.308743,0.156887,0,...,0,0,0,0,1,0,0,0,1,0
2,Nansukusa Sarah,fb94fff4-9217-470f-9e80-12621dea6191,2009-02-07,14.68,-3393780.01,265,-2.311839e+05,18.051771,0.156148,0,...,0,1,0,0,0,1,0,0,0,1
3,Bridget Namwalwa,ff0563dc-6006-4b4f-a88a-1e7b64adb040,2016-04-15,7.49,-8005741.43,262,-1.068857e+06,34.979973,0.136193,0,...,1,0,0,0,0,1,0,0,1,0
4,William William,a185cde9-4893-42c0-bcad-fbcf101d2a77,2013-03-15,10.58,-8434974.90,260,-7.972566e+05,24.574669,0.135615,0,...,0,0,0,0,0,1,1,0,0,0


## Convert the business duration column into ranges
psuedo code:
find the min value in duration column : e.g customer_data_encoded.min("")
find the max value in duration column
create ranges based on your judgeemnt. For exmaple if minimun is 1 and max is 10, then 3 ranges makes sense
create a new column in customer_data_encoded called business duration range and assign each row to it's range
e.g.: `customer name`` `uid``` `start`` `date`` `duration`` `duration range`` `other columens...``
        James WIlliam   ccds    2013-234          10.58      7 - 10 years


After that, one hot encode the  `duration range` column so that we have sth like:
`customer name`` `uid``` `start`` `date`` `duration`` `duration range_0 - 3 years`` `duration range_3 - 7 years` `duration range_7 - 10 years`
James WIlliam   ccds    2013-234          10.58       0                             0                                 1

In [9]:
# 1. Find the minimum and maximum values in the "business duration (years)" column
min_duration = customer_data_encoded['business duration (years)'].min()
max_duration = customer_data_encoded['business duration (years)'].max()

print("min is " + str(min_duration))
print("max is " + str(max_duration))
# 1. Define the specific bins
bins = [0, 5, 10, 15, 20.01]  # 20.01 is included to ensure businesses with exactly 20.01 years are also categorized

# 2. Create the labels
labels = ['0 - 5 years', 
          '5 - 10 years', 
          '10 - 15 years', 
          '15 - 20 years']  # Notice the last label is '15 - 20 years', not '15 - 20.01 years' for display purposes

# 3. Create a new column called "business duration range" in the original dataframe and assign each row its respective range
customer_data_encoded['business duration range'] = pd.cut(customer_data_encoded['business duration (years)'], bins=bins, labels=labels, right=False, include_lowest=True)
duration_dummies = pd.get_dummies(customer_data_encoded['business duration range'], prefix='duration range')

duration_dummies = duration_dummies.astype(int)


# Extract rows for our own analysis
business_duration_ranges_df = customer_data_encoded.groupby('business duration range').size().reset_index(name='Number of Businesses')
business_duration_ranges_df.head()

# Reorder the columns
desired_order = ['Customer Name', 'uid', 'business start date', 'business duration (years)', 'business duration range'] + [col for col in customer_data_encoded if col not in ['Customer Name', 'uid', 'business start date', 'business duration (years)', 'business duration range']]
customer_data_encoded = customer_data_encoded[desired_order]

customer_data_encoded.head()





min is 0.07
max is 20.06


,Customer Name,uid,business start date,business duration (years),business duration range,Net Profit,Transaction Volume,Net Profit Per Year,Transaction Volume Per Year,Normalized Performance Index,...,location_Kisoro,location_Lira,location_Mbale,location_Mbarara,gender_Female,gender_Male,education level_Primary,education level_Secondary,education level_University,education level_Vocational
0,James William,cca67301-ef3c-4496-b8fd-31847b7edd88,2013-02-25,10.63,10 - 15 years,3513483.23,294,3.305252e+05,27.657573,0.183499,...,0,0,0,0,1,0,0,0,1,0
1,James Nansukusa,7a1c5730-b8e7-4774-8746-90a317b73a82,2009-02-22,14.64,10 - 15 years,-3204307.06,107,-2.188734e+05,7.308743,0.156887,...,0,0,0,0,1,0,0,0,1,0
2,Nansukusa Sarah,fb94fff4-9217-470f-9e80-12621dea6191,2009-02-07,14.68,10 - 15 years,-3393780.01,265,-2.311839e+05,18.051771,0.156148,...,0,1,0,0,0,1,0,0,0,1
3,Bridget Namwalwa,ff0563dc-6006-4b4f-a88a-1e7b64adb040,2016-04-15,7.49,5 - 10 years,-8005741.43,262,-1.068857e+06,34.979973,0.136193,...,1,0,0,0,0,1,0,0,1,0
4,William William,a185cde9-4893-42c0-bcad-fbcf101d2a77,2013-03-15,10.58,10 - 15 years,-8434974.90,260,-7.972566e+05,24.574669,0.135615,...,0,0,0,0,0,1,1,0,0,0


## Encode business ranges column 

In [10]:
duration_dummies = pd.get_dummies(customer_data_encoded.get('business duration range', pd.Series()), prefix='duration range').astype(int)
customer_data_encoded = pd.concat([customer_data_encoded, duration_dummies], axis=1)
customer_data_encoded_range_dropped = customer_data_encoded.drop('business duration range', axis=1)
customer_data_encoded_range_dropped.head()





,Customer Name,uid,business start date,business duration (years),Net Profit,Transaction Volume,Net Profit Per Year,Transaction Volume Per Year,Normalized Performance Index,business sector_Agriculture,...,gender_Female,gender_Male,education level_Primary,education level_Secondary,education level_University,education level_Vocational,duration range_0 - 5 years,duration range_5 - 10 years,duration range_10 - 15 years,duration range_15 - 20 years
0,James William,cca67301-ef3c-4496-b8fd-31847b7edd88,2013-02-25,10.63,3513483.23,294,3.305252e+05,27.657573,0.183499,1,...,1,0,0,0,1,0,0,0,1,0
1,James Nansukusa,7a1c5730-b8e7-4774-8746-90a317b73a82,2009-02-22,14.64,-3204307.06,107,-2.188734e+05,7.308743,0.156887,0,...,1,0,0,0,1,0,0,0,1,0
2,Nansukusa Sarah,fb94fff4-9217-470f-9e80-12621dea6191,2009-02-07,14.68,-3393780.01,265,-2.311839e+05,18.051771,0.156148,0,...,0,1,0,0,0,1,0,0,1,0
3,Bridget Namwalwa,ff0563dc-6006-4b4f-a88a-1e7b64adb040,2016-04-15,7.49,-8005741.43,262,-1.068857e+06,34.979973,0.136193,0,...,0,1,0,0,1,0,0,1,0,0
4,William William,a185cde9-4893-42c0-bcad-fbcf101d2a77,2013-03-15,10.58,-8434974.90,260,-7.972566e+05,24.574669,0.135615,0,...,0,1,1,0,0,0,0,0,1,0


## Create transactions volume per year range column

In [11]:
# 1. Find the minimum and maximum transactions in the "Transaction Volume Per Year" column
min_transaction = customer_data_encoded_range_dropped['Transaction Volume Per Year'].min()
max_transaction = customer_data_encoded_range_dropped['Transaction Volume Per Year'].max()

print("min is " + str(min_transaction))
print("max is " + str(max_transaction))
bins = (0, 2000,4000,6000,8000,11000)

# 2. Create the labels
labels = ['0 - 2000', 
          '2000 - 4000', 
          '4000 - 6000',
          '6000 - 8000', 
          '8000 - 11000'] 

customer_data_encoded_range_dropped['transaction volume range'] = pd.cut(customer_data_encoded_range_dropped['Transaction Volume Per Year'], bins=bins, labels=labels, right=False, include_lowest=True)


customer_data_encoded_range_dropped.head()

min is 2.5062656641604013
max is 3071.428571428571


,Customer Name,uid,business start date,business duration (years),Net Profit,Transaction Volume,Net Profit Per Year,Transaction Volume Per Year,Normalized Performance Index,business sector_Agriculture,...,gender_Male,education level_Primary,education level_Secondary,education level_University,education level_Vocational,duration range_0 - 5 years,duration range_5 - 10 years,duration range_10 - 15 years,duration range_15 - 20 years,transaction volume range
0,James William,cca67301-ef3c-4496-b8fd-31847b7edd88,2013-02-25,10.63,3513483.23,294,3.305252e+05,27.657573,0.183499,1,...,0,0,0,1,0,0,0,1,0,0 - 2000
1,James Nansukusa,7a1c5730-b8e7-4774-8746-90a317b73a82,2009-02-22,14.64,-3204307.06,107,-2.188734e+05,7.308743,0.156887,0,...,0,0,0,1,0,0,0,1,0,0 - 2000
2,Nansukusa Sarah,fb94fff4-9217-470f-9e80-12621dea6191,2009-02-07,14.68,-3393780.01,265,-2.311839e+05,18.051771,0.156148,0,...,1,0,0,0,1,0,0,1,0,0 - 2000
3,Bridget Namwalwa,ff0563dc-6006-4b4f-a88a-1e7b64adb040,2016-04-15,7.49,-8005741.43,262,-1.068857e+06,34.979973,0.136193,0,...,1,0,0,1,0,0,1,0,0,0 - 2000
4,William William,a185cde9-4893-42c0-bcad-fbcf101d2a77,2013-03-15,10.58,-8434974.90,260,-7.972566e+05,24.574669,0.135615,0,...,1,1,0,0,0,0,0,1,0,0 - 2000


## Encode transactions volume per year range

In [12]:
duration_dummies = pd.get_dummies(customer_data_encoded_range_dropped.get('transaction volume range', pd.Series()), prefix='transaction range').astype(int)
customer_data_encoded_range_dropped = pd.concat([customer_data_encoded_range_dropped, duration_dummies], axis=1)
customer_data_encoded_range_dropped = customer_data_encoded_range_dropped.drop('transaction volume range', axis=1)
customer_data_encoded_range_dropped.head()


,Customer Name,uid,business start date,business duration (years),Net Profit,Transaction Volume,Net Profit Per Year,Transaction Volume Per Year,Normalized Performance Index,business sector_Agriculture,...,education level_Vocational,duration range_0 - 5 years,duration range_5 - 10 years,duration range_10 - 15 years,duration range_15 - 20 years,transaction range_0 - 2000,transaction range_2000 - 4000,transaction range_4000 - 6000,transaction range_6000 - 8000,transaction range_8000 - 11000
0,James William,cca67301-ef3c-4496-b8fd-31847b7edd88,2013-02-25,10.63,3513483.23,294,3.305252e+05,27.657573,0.183499,1,...,0,0,0,1,0,1,0,0,0,0
1,James Nansukusa,7a1c5730-b8e7-4774-8746-90a317b73a82,2009-02-22,14.64,-3204307.06,107,-2.188734e+05,7.308743,0.156887,0,...,0,0,0,1,0,1,0,0,0,0
2,Nansukusa Sarah,fb94fff4-9217-470f-9e80-12621dea6191,2009-02-07,14.68,-3393780.01,265,-2.311839e+05,18.051771,0.156148,0,...,1,0,0,1,0,1,0,0,0,0
3,Bridget Namwalwa,ff0563dc-6006-4b4f-a88a-1e7b64adb040,2016-04-15,7.49,-8005741.43,262,-1.068857e+06,34.979973,0.136193,0,...,0,0,1,0,0,1,0,0,0,0
4,William William,a185cde9-4893-42c0-bcad-fbcf101d2a77,2013-03-15,10.58,-8434974.90,260,-7.972566e+05,24.574669,0.135615,0,...,0,0,0,1,0,1,0,0,0,0


Drop the transaction range Column, transaction vol per year, net profit per year, transaction volume colume, net profit columns.  Name the new data frame, transaction dropped data - encoded_with_transactions

In [13]:

dropped_data_encoded_with_transactions = customer_data_encoded_range_dropped.drop(columns=['Transaction Volume Per Year', 'Net Profit Per Year', 'Transaction Volume', 'Net Profit'])
dropped_data_encoded_with_transactions.head()

,Customer Name,uid,business start date,business duration (years),Normalized Performance Index,business sector_Agriculture,business sector_Fisheries,business sector_Hospitality,business sector_IT,business sector_Manufacturing,...,education level_Vocational,duration range_0 - 5 years,duration range_5 - 10 years,duration range_10 - 15 years,duration range_15 - 20 years,transaction range_0 - 2000,transaction range_2000 - 4000,transaction range_4000 - 6000,transaction range_6000 - 8000,transaction range_8000 - 11000
0,James William,cca67301-ef3c-4496-b8fd-31847b7edd88,2013-02-25,10.63,0.183499,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,James Nansukusa,7a1c5730-b8e7-4774-8746-90a317b73a82,2009-02-22,14.64,0.156887,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
2,Nansukusa Sarah,fb94fff4-9217-470f-9e80-12621dea6191,2009-02-07,14.68,0.156148,0,1,0,0,0,...,1,0,0,1,0,1,0,0,0,0
3,Bridget Namwalwa,ff0563dc-6006-4b4f-a88a-1e7b64adb040,2016-04-15,7.49,0.136193,0,0,0,0,1,...,0,0,1,0,0,1,0,0,0,0
4,William William,a185cde9-4893-42c0-bcad-fbcf101d2a77,2013-03-15,10.58,0.135615,0,0,1,0,0,...,0,0,0,1,0,1,0,0,0,0


## Export encoded data in to csv file

In [14]:
dropped_data_encoded_with_transactions.to_csv('encoded_data.csv', index=False)


In [16]:
dropped_data_encoded_with_transactions.head(1)

,Customer Name,uid,business start date,business duration (years),Normalized Performance Index,business sector_Agriculture,business sector_Fisheries,business sector_Hospitality,business sector_IT,business sector_Manufacturing,...,education level_Vocational,duration range_0 - 5 years,duration range_5 - 10 years,duration range_10 - 15 years,duration range_15 - 20 years,transaction range_0 - 2000,transaction range_2000 - 4000,transaction range_4000 - 6000,transaction range_6000 - 8000,transaction range_8000 - 11000
0,James William,cca67301-ef3c-4496-b8fd-31847b7edd88,2013-02-25,10.63,0.183499,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
